# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

#     Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.

#     Create a heat map that displays the humidity for every city from the part I of the homework.
#     Narrow down the DataFrame to find your ideal weather condition. For example: 
# ** A max temperature lower than 27 degrees but higher than 21. 
# ** Wind speed less than 15 kph. 
# ** Zero cloudiness. 
# ** Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. 

# ** Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

#     Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
#     Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# As final considerations:

#     You must complete your analysis using a Jupyter notebook.
#     You must use the Matplotlib or Pandas plotting libraries.
#     For Part I, you must include a written description of three observable trends based on the data.
#     For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
#     You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
#     For max intensity in the heat map, try setting it to the highest humidity found in the data set.


In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

pd.options.mode.chained_assignment = None  
# default='warn'

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
draft_cities_df_path = "Resources/global_power_plant_database.csv"
draft_cities_df = pd.read_csv("../output_data/cities.csv")
draft_cities_df = draft_cities_df.dropna()
draft_cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [20]:
# plot heatmap
heatmap_layer = gmaps.heatmap_layer(draft_cities_df[['Lat', 'Lng']].values, weights=draft_cities_df['Humidity'].values, dissipating=True)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
draft_cities_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
10,10,aykhal,90,RU,1558378756,48,65.95,111.51,39.20,11.18


In [22]:
# Carlys Personal Criteria
# Humidity less than 45%
# max temperature lower than 28 degrees but higher than 18.
# wind speed less than 15 kph. 
# ** Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. 

holiday_destinations_df = draft_cities_df[(draft_cities_df["Humidity"] < 45) &
                                            (draft_cities_df["Wind Speed"] < 15) & 
                                            (draft_cities_df["Max Temp"] < 84) & 
                                            (draft_cities_df["Max Temp"] > 64)]
holiday_destinations_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
75,75,saint george,40,GR,1558378768,33,39.45,22.34,75.20,11.41
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
260,260,pahrump,75,US,1558378834,41,36.21,-115.98,68.00,11.50
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50
311,311,balkanabat,99,TM,1558378846,41,39.51,54.36,79.80,6.69
340,340,la libertad,0,PE,1558378855,36,-16.38,-71.56,64.40,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# Cretae a new column to populate the hotel name into

holiday_destinations_df = holiday_destinations_df
holiday_destinations_df["Hotel Name"] = " "
holiday_destinations_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,
75,75,saint george,40,GR,1558378768,33,39.45,22.34,75.20,11.41,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,
260,260,pahrump,75,US,1558378834,41,36.21,-115.98,68.00,11.50,
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50,
311,311,balkanabat,99,TM,1558378846,41,39.51,54.36,79.80,6.69,
340,340,la libertad,0,PE,1558378855,36,-16.38,-71.56,64.40,8.05,


In [24]:
#holiday_destinations_df

# find nearest hotel

def find_hotel(lat, lng, radius):
    """
    Finds the nearest hotel to the given coordinates.
    """
    # set up a parameters dictionary
    params = {"location": f"{lat},{lng}",
                "radius": radius,
                "type": "lodging",
                "key": g_key}

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    # print(json.dumps(places_data, indent=4, sort_keys=True))

    # Print the name and address of the first hotel that appears
    hotel_name = "None"
    if len(places_data["results"]) > 0:
        hotel_name = places_data["results"][0]["name"]
    return hotel_name

In [25]:
# find hotel name for each city and Add it to the dataframe
holiday_destinations_df["Hotel Name"] = holiday_destinations_df.apply(lambda row: find_hotel(row["Lat"], row["Lng"], radius=5000), axis=1)

In [26]:
# Display final holiday destinations dataframe
holiday_destinations_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,لی پوش jean center
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Hotel San Isidro Oficial
75,75,saint george,40,GR,1558378768,33,39.45,22.34,75.20,11.41,MX PARK KYPARISSOS
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,Ramada by Wyndham Moses Lake
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,Post And Telecommunication Hotel
260,260,pahrump,75,US,1558378834,41,36.21,-115.98,68.00,11.50,Best Western Pahrump Oasis
296,296,khash,32,IR,1558378844,43,28.22,61.21,68.28,12.50,داروخانه دامپزشکی سارانی
311,311,balkanabat,99,TM,1558378846,41,39.51,54.36,79.80,6.69,Nebitchi Hotel
340,340,la libertad,0,PE,1558378855,36,-16.38,-71.56,64.40,8.05,Hotel La cuesta de Cayma


In [40]:
# TEMPLATE TO PLOT THE PIN LAYER ON THE HEATMAP WHICH INCLUDES THE HOTEL NAME, CITY AND COUNTRY
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name: {Hotel Name}
City: {City}
Country: {Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in holiday_destinations_df.iterrows()]
locations = holiday_destinations_df[["Lat", "Lng"]]

In [41]:
# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

gmaps.configure(api_key=g_key) 
fig = gmaps.figure()
# plot heatmap
heatmap_layer = gmaps.heatmap_layer(holiday_destinations_df[['Lat', 'Lng']].values, weights=holiday_destinations_df['Humidity'].values, dissipating=True)

fig.add_layer(heatmap_layer)
# add hotel name, city, and country to the heatmap
fig.add_layer(gmaps.marker_layer(holiday_destinations_df[['Lat', 'Lng']].values, hotel_info))

fig

Figure(layout=FigureLayout(height='420px'))